In [1]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()

In [2]:
import os
import torch
import pickle
import logging
import numpy as np
import pandas as pd
import torch.nn as nn
import seaborn as sns
import torch.optim as optim
import matplotlib.pyplot as plt

from torch import nn
from tqdm import tqdm
from enum import Enum
from pprint import pprint
from itertools import chain
from dataclasses import dataclass
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
from concurrent.futures import ThreadPoolExecutor
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, Dataset, random_split
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)


In [3]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
%cd ..
!mamba install TA-Lib
import talib

zsh:1: command not found: wget
tar: Error opening archive: Failed to open 'ta-lib-0.4.0-src.tar.gz'
[Errno 2] No such file or directory: 'ta-lib'
/Users/collin/PycharmProjects/dl-quant/notebooks
zsh:1: no such file or directory: ./configure
make: *** No targets specified and no makefile found.  Stop.
make: *** No rule to make target `install'.  Stop.
/Users/collin/PycharmProjects/dl-quant

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔

In [4]:
!pip install alpaca-py
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data import StockBarsRequest, TimeFrame

In [5]:
API_KEY = "PK3D0P0EF5NVU7LKHY76"
API_SECRET = "X2kmdCqfYzGxaCYG2C3UhQ9DqHT9bYhYUhXM2g6G"

In [6]:
sp_tickers = [
    "AAPL", "MSFT", "GOOGL", "GOOG", "AMZN", "TSLA", "FB", "BRK.B", "JPM", "JNJ",
    "NVDA", "V", "PG", "MA", "UNH", "PYPL", "HD", "DIS", "BAC", "CMCSA",
    "TMO", "ADBE", "XOM", "CRM", "INTC", "PEP", "VZ", "MRK", "NFLX", "PFE",
    "KO", "ABBV", "T", "CSCO", "ABBV", "CVX", "NKE", "WMT", "COST", "MCD",
    "C", "ABT", "MRNA", "QCOM", "GILD", "TGT", "LOW", "BMY", "HON", "DHR",
    "SBUX", "AVGO", "PM", "ORCL", "IBM", "MS", "UPS", "CHTR", "CAT", "LMT",
    "NEE", "GS", "MMM", "AMGN", "MO", "AMAT", "TXN", "BKNG", "LLY", "ADP",
    "NOW", "ISRG", "ANTM", "TFC", "CVS", "VRTX", "USB", "DUK", "TMUS", "CCI",
    "PLD", "SPGI", "ECL", "GPN", "BLK", "GM", "BDX", "ICE", "SO", "CI", "ETN",
    "FDX", "GD", "LRCX", "EOG", "APD", "SHW", "PNC", "MET", "NSC", "AON",
    "ADSK", "CL", "MMC", "CB", "TROW", "MCO", "EQIX", "TMX", "TDG", "COF",
    "BAX", "SYK", "PSX", "BIIB", "FIS", "ITW", "WM", "DG", "AEP", "FD", "EMR",
    "LHX", "SNP", "AZO", "ROST", "NOC", "AIG", "KR", "ESS", "VRTX", "SCHW",
    "CTSH", "CTAS", "STZ", "FE", "AFL", "WBA", "AMP", "TROW", "SO", "BLL",
    "WELL", "MKC", "EOG", "KMB", "DUK", "AEP", "DTE", "CBRE", "EXC", "BKR",
    "PPL", "WEC", "AWK", "ALLE", "EIX", "VRSN", "D", "WLTW", "UDR", "LDOS",
    "HLT", "FLT", "VTRS", "PXD", "EW", "ATO", "CDW", "PAYC", "FTNT", "OTIS",
    "TDY", "CDNS", "MSCI", "ZBRA", "TWTR", "FTV", "KEYS", "STE", "ANSS", "SNPS",
    "ZTS", "LBRDK", "IQV", "MKTX", "RMD", "MXIM", "CRL", "FRC", "CPRT", "IDXX",
    "JKHY", "AJG", "PEAK", "ESS", "CTXS", "URI", "SWK", "DLR", "VNO", "FAST",
    "LNT", "BR", "AIZ", "ALGN", "UAA", "MLM", "TTWO", "CPB", "CINF", "VFC",
    "KSU", "FTNT", "TDY", "LW", "WAB", "HRL", "AME", "ATO", "HIG", "HBAN",
    "ROL", "ANET", "WRB", "NUE", "UDR", "ARE", "RSG", "FTV", "NVR", "J",
    "MKTX", "FLT", "ABMD", "AKAM", "AVB", "RHI", "SNPS", "ZBRA", "IDXX", "CARR",
    "RE", "LYV", "CBRE", "CMI", "DVA", "LVS", "CTLT", "DXCM", "CERN", "AAL",
    "COG", "SIVB", "KMX", "HST", "NTRS", "CHRW", "ZION", "TPR", "APA", "CBOE",
    "VTR", "DVN", "LH", "NI", "WY", "CMS", "GL", "EXPE", "LYB", "O", "VLO",
    "NCLH", "FFIV", "L", "TDG", "BBWI", "HCA", "UNM", "JNPR", "MAS", "WDC",
    "BKR", "NOV", "CNP", "MGM", "XRX", "PVH", "UHS", "VNT", "APA", "AES",
    "DXC", "GPS", "BRX", "PRGO", "NRG", "RJF", "FLS", "F", "TDG", "HOLX", "PWR",
    "URI", "AAP", "NLOK", "HP", "NWL", "MHK", "VAR", "LEG", "WELL", "COTY",
    "FLR", "ARNC", "FANG", "KEY", "LKQ", "SEE", "FLIR", "HFC", "NRG", "BHF",
    "DISCK", "DISCA", "FMC", "BEN", "PKG", "AJG", "MHK", "IVZ", "CF", "NWSA",
    "NWS", "PBCT", "XRX", "EXR", "WRK", "TAP", "HFC", "DISH", "PKG", "OXY",
    "IVZ", "UA", "DISCK", "DISCA", "NWL", "FOX", "MHK", "HPE", "NOV", "TAP",
    "KSS", "BEN", "EXR", "PBCT", "NWSA", "NWS", "UA", "DISH", "FOX", "OXY",
    "ADS", "KHC", "MOS", "PVH", "DXC", "UA", "NBL", "ADS", "KHC", "PVH", "DXC",
    "MOS", "SYF", "ZBH", "NCLH", "BRX", "MRO", "GPS", "L", "UAA", "FLS", "FLR",
    "RJF", "RJF", "JNPR", "UHS", "HCA", "BBWI", "HP", "NLOK", "AAP", "HOLX",
    "PWR", "ARNC", "COTY", "LEG", "VAR", "MHK", "FANG", "FLIR", "SEE", "LKQ",
    "KEY", "BHF", "FMC", "DISCK", "DISCA", "BEN", "PKG", "IVZ", "CF", "NWSA",
    "NWS", "PBCT", "XRX", "EXR", "WRK", "TAP", "HFC", "DISH", "PKG", "OXY",
    "IVZ", "UA", "FOX", "NWL", "MHK", "HPE", "NOV", "TAP", "KSS", "BEN", "EXR",
    "PBCT", "NWSA", "NWS", "UA", "DISH", "FOX", "OXY", "ADS", "KHC", "MOS",
    "PVH", "DXC", "UA", "NBL", "ADS", "KHC", "PVH", "DXC", "MOS", "SYF", "ZBH",
    "NCLH", "BRX", "MRO", "GPS", "L", "UAA", "FLS", "FLR", "RJF", "JNPR", "UHS",
    "HCA", "BBWI", "HP", "NLOK", "AAP", "HOLX", "PWR", "ARNC", "COTY", "LEG",
    "VAR", "MHK", "FANG", "FLIR", "SEE", "LKQ", "KEY", "BHF", "FMC", "DISCK",
    "DISCA", "BEN", "PKG", "IVZ", "CF", "NWSA", "NWS", "PBCT", "XRX", "EXR",
    "WRK", "TAP", "HFC", "DISH", "PKG", "OXY", "IVZ", "UA", "FOX", "NWL", "MHK",
    "HPE", "NOV", "TAP", "KSS", "BEN", "EXR", "PBCT", "NWSA", "NWS", "UA",
    "DISH", "FOX", "OXY", "ADS", "KHC", "MOS", "PVH", "DXC", "UA", "NBL", "ADS",
    "KHC", "PVH", "DXC", "MOS", "SYF", "ZBH", "NCLH", "BRX", "MRO", "GPS", "L",
    "UAA", "FLS", "FLR", "RJF", "JNPR", "UHS", "HCA", "BBWI", "HP", "NLOK",
    "AAP", "HOLX", "PWR", "ARNC", "COTY", "LEG", "VAR", "MHK", "FANG", "FLIR",
    "SEE", "LKQ", "KEY", "BHF", "FMC", "DISCK", "DISCA", "BEN", "PKG", "IVZ",
    "CF", "NWSA", "NWS", "PBCT", "XRX", "EXR", "WRK", "TAP", "HFC", "DISH",
    "PKG", "OXY", "IVZ", "UA", "FOX", "NWL", "MHK", "HPE", "NOV", "TAP", "KSS",
    "BEN", "EXR", "PBCT", "NWSA", "NWS", "UA", "DISH", "FOX", "OXY", "ADS",
    "KHC", "MOS", "PVH", "DXC", "UA", "NBL", "ADS", "KHC", "PVH", "DXC", "MOS",
    "SYF", "ZBH", "NCLH", "BRX", "MRO", "GPS", "L", "UAA", "FLS", "FLR", "RJF",
    "JNPR", "UHS", "HCA", "BBWI", "HP", "NLOK", "AAP", "HOLX", "PWR", "ARNC",
    "COTY", "LEG", "VAR", "MHK", "FANG", "FLIR", "SEE", "LKQ", "KEY", "BHF",
    "FMC", "DISCK", "DISCA", "BEN", "PKG", "IVZ", "CF", "NWSA", "NWS", "PBCT",
    "XRX", "EXR", "WRK", "TAP", "HFC", "DISH", "PKG", "OXY", "IVZ", "UA", "FOX",
    "NWL", "MHK", "HPE", "NOV", "TAP", "KSS", "BEN", "EXR", "PBCT", "NWSA",
    "NWS", "UA", "DISH", "FOX", "OXY", "ADS", "KHC", "MOS", "PVH", "DXC", "UA",
    "NBL", "ADS", "KHC", "PVH", "DXC", "MOS", "SYF", "ZBH", "NCLH", "BRX",
    "MRO", "GPS", "L", "UAA", "FLS", "FLR", "RJF", "JNPR", "UHS", "HCA", "BBWI",
    "HP", "NLOK", "AAP", "HOLX", "PWR", "ARNC", "COTY", "LEG", "VAR", "MHK",
    "FANG", "FLIR", "SEE", "LKQ", "KEY", "BHF", "FMC", "DISCK", "DISCA", "BEN",
    "PKG", "IVZ", "CF", "NWSA", "NWS", "PBCT", "XRX", "EXR", "WRK", "TAP", "HFC",
    "DISH", "PKG", "OXY", "IVZ", "UA", "FOX", "NWL", "MHK", "HPE", "NOV", "TAP",
    "KSS", "BEN", "EXR", "PBCT"
]
len(sp_tickers)

754

In [7]:
class Vec(nn.Module):
    def __init__(self, input_size):
        super(Vec, self).__init__()
        self.lstm1 = nn.LSTM(input_size, 32, batch_first=True)  # Reduced complexity
        self.dropout1 = nn.Dropout(0.5)  # Added dropout
        self.lstm2 = nn.LSTM(32, 32, batch_first=True)  # Reduced complexity
        self.dropout2 = nn.Dropout(0.5)  # Added dropout
        self.linear = nn.Linear(32, 15)

    def forward(self, input, future=0):
        outputs = []
        h_t, _ = self.lstm1(input)
        h_t = self.dropout1(h_t)
        h_t2, _ = self.lstm2(h_t)
        h_t2 = self.dropout2(h_t2)
        output = self.linear(h_t2[:, -1, :])
        outputs += [output]
        for i in range(future):  # if we should predict the future
            h_t, _ = self.lstm1(output.unsqueeze(1))
            h_t2, _ = self.lstm2(h_t)
            output = self.linear(h_t2[:, -1, :])
            outputs += [output]
        outputs = torch.cat(outputs, dim=1)
        return outputs


class MetaLabeler(nn.Module):
    """
    A transformer model for determining the magnitude of Vec's and RVec's proposed trades. MetaLabeler is trained
    upon Vec's and RVec's predictions.
    """

    def __init__(self, *args, **kwargs):
        super(MetaLabeler, self).__init__()


In [8]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, input_window_size=60, output_window_size=15):
        self.data = data
        self.input_window_size = input_window_size
        self.output_window_size = output_window_size
        self.total_window_size = input_window_size + output_window_size

    def __len__(self):
        print(self.data.shape[1], self.total_window_size)
        print(self.data.shape[1] - self.total_window_size + 1)
        return self.data.shape[1] - self.total_window_size + 1

    def __getitem__(self, index):
        # Loop through each stock and yield a window for the given index
        input_batch = []
        output_batch = []
        for stock_index in range(self.data.shape[0]):
            input_window = self.data[stock_index, :, index: index + self.input_window_size]
            output_window = self.data[stock_index, :, index + self.input_window_size: index + self.total_window_size]
            input_batch.append(input_window)
            output_batch.append(output_window)

        return torch.stack(input_batch), torch.stack(output_batch)


In [9]:
class NullLogger:
    def __init__(self, *args, **kwargs):
        ...

    def info(self, *args, **kwargs):
        ...

    def debug(self, *args, **kwargs):
        ...

    def warning(self, *args, **kwargs):
        ...

    def error(self, *args, **kwargs):
        ...

In [10]:
def alpaca_time_series(
    stocks: list[str], start: datetime or str, end: datetime or str
) -> dict:
    """
    Retrieve data from the Alpaca API.
    """

    if isinstance(start, datetime):
        start = start.strftime("%Y-%m-%d %H:%M:%S")

    if isinstance(end, datetime):
        end = end.strftime("%Y-%m-%d %H:%M:%S")

    client = StockHistoricalDataClient(
        API_KEY,
        API_SECRET,
    )

    # url_override="https://data.alpaca.markets",

    params = StockBarsRequest(
        symbol_or_symbols=stocks,
        start=start,
        end=end,
        timeframe=TimeFrame.Minute,
    )

    return client.get_stock_bars(params).data


In [11]:
def fetch_data(
    years: float,
    max_workers: int = len(sp_tickers) // 2,
    log: logging.Logger = NullLogger(),
) -> list:
    # Retrieve data from the Alpaca API
    log.info("Retrieving data from the Alpaca API...")
    log.info(f"Maximum Worker-threads: {max_workers}")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(
                alpaca_time_series,
                [ticker],
                datetime.today() - timedelta(365 * years),
                datetime.today(),
            )
            for ticker in sp_tickers
        ]

    # Collecting results
    data: list = [future.result() for future in futures]
    log.info("Retrieved data from the Alpaca API.")

    return data


In [12]:
def stock_tensors(stock_data) -> torch.Tensor:
    """
    Applies the `generate_indicators` function to each stock in the input dictionary, pads each resulting tensor to the
    maximum dimension among them, and then stacks them into a single tensor.

    :param stock_data: A dictionary where the keys are stock names and the values are time series data for each stock.

    :returns torch.Tensor: A tensor that is the result of stacking the padded tensors obtained from each stock's time
    series data.
    """
    indicator_tensors_list = []

    if type(stock_data) is dict:
        indicator_tensors_list = [
            indicators(stock) for stock in list(stock_data.values())
        ]
    else:
        # Aggregate all dictionaries contained within the list.
        aggregated_data = {}
        for stock_dict in stock_data:
            for symbol, data in stock_dict.items():
                aggregated_data[symbol] = data

        indicator_tensors_list = [
            indicators(stock) for stock in list(aggregated_data.values())
        ]

    max_dim = max([stock.shape[1] for stock in indicator_tensors_list])
    padded_indicator_tensors = [
        torch.nn.functional.pad(
            stock, (max_dim - stock.shape[1], 0), mode="constant", value=0.0
        )
        for stock in indicator_tensors_list
    ]
    return torch.stack(padded_indicator_tensors, dim=0)


In [13]:
def indicators(time_series: list, _indicators: list = None) -> torch.Tensor:
    """
    Generate technical indicators for a given time series.
    """

    closes = np.array([bar.close for bar in time_series], dtype=np.float64)
    opens = np.array([bar.open for bar in time_series], dtype=np.float64)
    highs = np.array([bar.high for bar in time_series], dtype=np.float64)
    lows = np.array([bar.low for bar in time_series], dtype=np.float64)
    volumes = np.array([bar.volume for bar in time_series], dtype=np.float64)

    indicator_functions = {
        "SMA": lambda x=closes, interval=5: talib.SMA(x, timeperiod=interval),
        "EMA": lambda x=closes, interval=12: talib.EMA(x, timeperiod=interval),
        "WMA": lambda x=closes, interval=12: talib.WMA(x, timeperiod=interval),
        # "BBANDS" has shape (3, t), as opposed to (t)
        # "BBANDS": lambda x=closes, interval=20: talib.BBANDS(
        #     x, timeperiod=interval, nbdevup=1.5, nbdevdn=1.5
        # ),
        "CCI": lambda interval=20: talib.CCI(highs, lows, closes, timeperiod=interval),
        "ROC": lambda x=closes, interval=12: talib.ROC(x, timeperiod=interval),
        "RSI": lambda x=closes, interval=14: talib.RSI(x, timeperiod=interval),
        # "STOCH" has shape (2, t), as opposed to (t)
        # "STOCH": lambda fk=5, sk=3: talib.STOCH(
        #     highs, lows, closes, fastk_period=fk, slowk_period=sk
        # ),
        "MFI": lambda interval=14: talib.MFI(
            highs, lows, closes, volumes, timeperiod=interval
        ),
        "SAR": lambda a=0.02, m=0.2, interval=0: talib.SAR(
            highs, lows, acceleration=a, maximum=m
        ),
    }

    indicator_tensors = []

    if not _indicators:
        for func in indicator_functions.values():
            result = np.array(func())
            indicator_tensor = torch.tensor(result, dtype=torch.float64)
            indicator_tensors.append(indicator_tensor)
    else:
        for indicator in _indicators:
            result = np.array(indicator_functions[indicator]())
            indicator_tensor = torch.tensor(result, dtype=torch.float64)
            indicator_tensors.append(indicator_tensor)

    indicator_time_series = torch.stack(indicator_tensors, dim=0)

    indicator_time_series = torch.nan_to_num(
        indicator_time_series,
        nan=0.0,
        posinf=indicator_time_series.max(),
        neginf=indicator_time_series.min(),
    )

    # indicator_time_series = torch.norm(
    #     indicator_time_series + 1.0
    # )  # Add 1 to avoid norm(0)

    return indicator_time_series


In [14]:
def create_windows(
    data,
    input_window_size=60,
    output_window_size=15,
    log: logging.Logger = NullLogger(),
) -> tuple[torch.Tensor, torch.Tensor]:
    """
    Create input and output windows for the Vec model.

    :param data: A tensor of shape (n, t, d) where n is the number of stocks, t is the number of time steps, and d is the
    number of technical indicators.
    :param input_window_size: The number of time steps to use as input.
    :param output_window_size: The number of time steps to use as output.
    :param log: A logger object.
    :return: A tuple of tensors where the first tensor is the input data and the second tensor is the output data.
    """
    input_data, output_data = [], []
    insufficient_data_count = 0

    # Loop over each stock
    for stock_index in range(data.shape[0]):
        stock_data = data[stock_index]  # Get data for the current stock
        stock_input, stock_output = [], []

        # Check if there are enough time steps for the current stock
        if stock_data.shape[1] >= input_window_size + output_window_size:
            # Create windows for the current stock
            for i in range(
                stock_data.shape[1] - input_window_size - output_window_size + 1
            ):
                input_window = stock_data[:, i : i + input_window_size]
                output_window = stock_data[
                    :,
                    i + input_window_size : i + input_window_size + output_window_size,
                ]
                stock_input.append(input_window)
                stock_output.append(output_window)

            input_data.extend(stock_input)
            output_data.extend(stock_output)
        else:
            insufficient_data_count += 1
            print(
                f"Insufficient data for stock index {stock_index} with time steps: {stock_data.shape[1]}."
            )

        print(
            f"Stock index {stock_index}: Created {len(stock_input)} input windows and {len(stock_output)} output windows."
        )

    if input_data and output_data:
        return torch.stack(input_data, dim=0), torch.stack(output_data, dim=0)
    else:
        print(
            f"No windows were created. Insufficient data stocks count: {insufficient_data_count}."
        )
        return torch.tensor([]), torch.tensor([])


In [15]:
time_series = fetch_data(years=0.25)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
def train(
    time_series: list,
    normalized: bool = True,
    epochs: int = 1_000_000,
    log: logging.Logger = NullLogger(),
    batch_size: int = 32,
) -> None:
    """
    Train the Vec model.
    """

    if torch.cuda.is_available():
        device = torch.device("cuda")
        log.info("CUDA is available.")
    else:
        device = torch.device("cpu")
        log.info("CUDA is not available.")

    log.info("Generating technical indicators for each stock...")
    time_series_tensor = stock_tensors(time_series)
    log.info("Generated technical indicators for each stock.")

    log.info("Normalizing the data...")
    norm_data = torch.nan_to_num(
        torch.log(time_series_tensor.detach().clone() + 1.0), nan=0.0
    )
    log.info("Normalized the data.")

    # Prepare dataset
    print(time_series_tensor.size())
    dataset = TimeSeriesDataset(norm_data if normalized else time_series_tensor)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    # Prepare data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Instantiate the model
    transformer = Vec(dataset[0][0].shape[-1]).to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(transformer.parameters(), lr=0.001)

    graph_loss = []
    best_loss = float("inf")
    early_stopping_patience = 10
    patience_counter = 0

    for epoch in tqdm(range(epochs)):
        transformer.train()
        total_loss = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = transformer(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        transformer.eval()
        total_val_loss = 0
        with torch.no_grad():
            for X_val_batch, y_val_batch in val_loader:
                X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(
                    device
                )
                val_output = transformer(X_val_batch)
                val_loss = criterion(val_output, y_val_batch)
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter > early_stopping_patience:
                log.info(f"Early stopping triggered at epoch {epoch}.")
                break

        graph_loss.append(avg_train_loss)

    if not os.path.exists("plots"):
        os.mkdir("plots")

    plt.plot(graph_loss)
    plt.title("Loss vs. Epochs (Normalized)" if normalized else "Loss vs. Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.savefig("plots/loss.png")
    plt.clf()

    log.info("Saving transformer.pt...")
    if not os.path.exists("models"):
        os.mkdir("models")

    torch.save(transformer.state_dict(), "models/transformer.pt")
    log.info("Saved transformer.pt")


In [ ]:
train(time_series=time_series)